# Water abstraction - Drawdown prediction with the Theis solution for confined and unconfined aquifers
This notebook demonstrate the application of the Theis solution to predict the drawdown from pumping in an unconfined and a confined aquifer.

## Introduction

### General situation
We consider an unconfined aquifer with constant transmissivity. The solution is a correction of the Theis solution for confined aquifers: If a well is pumping water out of the aquifer, radial flow towards the well is induced. The following figure illustrates this situation.

<img src="FIGS/Flow_well_confined.png" width="400">

*Fig: Radial flow in a confined aquifer (Fetter, 1994)*

The calculate the hydraulic situation, the following simplified flow equation can be used. This equation accounts for 1D radial transient flow towards a fully penetrating well within a confined aquifer without further sinks and sources:

$$ \frac{\partial^2 h}{\partial r^2}+\frac{1}{r}\frac{\partial h}{\partial r}=\frac{S}{T}\frac{\partial h}{\partial t} $$

### Solution by Theis
Charles V. Theis presented a solution for this by deriving

$$ s(r,t)=\frac{Q}{4\pi T}W(u) $$

with the well function

$$ W(u) = \int_{u }^{+\infty} \frac{e^{-\tilde u}}{\tilde u}d\tilde u $$

and the dimensionless variable 

$$ u = \frac{Sr^2}{4Tt} $$

This equations are not easy to solve. Historically, values for the well function were provided by tables or as so called type-curve. The type-curve matching with experimental data for pumping test analysis can be considered as one of the basic hydrogeological methods.

### Correction for unconfined aquifers

Jacob (in Kruseman and de Ridder 1994) proposed an conrrection of the Theis drawdown to account for unconfined aquifers.

$$ s' = s - \frac{s^2}{2b} $$

With a reformulation, this allows to compute the drawdown of unconfined aquifers as

$$ s = b - b \sqrt{1 - \frac{2s'}{b}} $$


### Computation and visualization

However, modern computers provide an easier and more convenient way to solve the 1D radial flow equation based on the Theis approach. Subsequently, the Theis equation is solved with Python routines and you can predict the drawdown for a vertical cross section with the well in the center.

You can further modify the parameters (transmissivity _T_, storage _S_, abstraction rate _Q_). And you can look (and output) the drawdown for a specific distance from the well (x_search).

In [1]:
# (First, the necessary Python functions are initiated.)

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
from ipywidgets import *

# (Here the necessary functions like the well function $W(u)$ are defined. Later, those functions are used in the computation)
# Define a function, class, and object for Theis Well analysis

def well_function(u):
    return scipy.special.exp1(u)

def theis_u(T,S,r,t):
    u = r ** 2 * S / 4. / T / t
    return u

def theis_s(Q, T, u):
    s = Q / 4. / np.pi / T * well_function(u)
    return s

# (Here, the method computes the data for the well function. Those data can be used to generate a type curve.)
u_max = 1
r_max = 10000
u  = [u_max for x in range(r_max)]
u_inv  = [r_max/u_max for x in range(r_max)]
w_u  = [well_function(u_max/r_max) for x in range(r_max)]

def compute_s(T, S, t, Q, r):
    u = theis_u(T, S, r, t)
    s = theis_s(Q, T, u)
    return s

def compute_s_unconfined(T, SY, t, Q, r, b):
    S_u = SY*b
    u = theis_u(T, S_u, r, t)
    s = theis_s(Q, T, u)
    s_u = b - b * np.sqrt(1-2*s/b)
    return s_u

def plot_s (t, T, S, SY, Q, b, x_search, max_s, max_r):
    # Range of delta_h / delta_l values (hydraulic gradient)
    r = np.linspace(1, max_r, 200)
    r_neg = r * -1.0
    
    # Compute Q for each hydraulic gradient
    s  = compute_s(T, S, t, Q, r)
    s_u  = compute_s_unconfined(T, SY, t, Q, r, b)

    # Compute s for a specific point
    x_point = x_search
    y_point = compute_s(T, S, t, Q, x_search)
    
    # Plotting
    plt.figure(figsize=(10, 6))
    
    plt.plot(r, s, linewidth=1., color='b', label=r'drawdown prediction')
    plt.plot(r_neg, s, linewidth=0.5, color='g', linestyle='dashed')
    plt.plot(r, s_u, linewidth=0.5, color='g', linestyle='dashed')
    plt.plot(r_neg, s_u, linewidth=1, color='g',label=r'drawdown prediction unconfined')
    plt.fill_between(r,s,max_s, facecolor='lightblue')
    plt.fill_between(r_neg,s_u,max_s, facecolor='lightgreen')
    plt.xlim(-max_r, max_r)
    plt.ylim(max_s,0)
    plt.plot(x_point,y_point, marker='o', color='r',linestyle ='None', label='drawdown output') 
    plt.xlabel(r'Distance from the well in m', fontsize=14)
    plt.ylabel(r'Drawdown in m', fontsize=14)
    plt.title('Drawdown prediction with Theis', fontsize=16)
    plt.legend()
    plt.grid(True)
    plt.show()

    print("DRAWDOWN output:")
    print("Distance from the well (in m): %8.2f" %x_point)
    print('Drawdown at this distance (in m):  %5.2f' %y_point)
    print('Time (in sec): ',t)
    

interact(plot_s,
         t = widgets.FloatSlider(value=3600,min=60, max=86400, step=60,readout=True),
         T = widgets.FloatLogSlider(value=0.03,base=10,min=-6, max=4, step=0.1,readout=True,readout_format='.2e'),
         S = widgets.FloatLogSlider(value=0.003,base=10,min=-9, max=1, step=0.1,readout=True,readout_format='.2e'),
         Q = widgets.FloatSlider(value=0.005,min=0.001, max=1, step=0.001,readout=True,readout_format='5.3f'),
         b = widgets.FloatSlider(value=20,min=1, max=100, step=0.01,readout=True,readout_format='5.3f'),
         SY = widgets.FloatSlider(value=0.25,min=0.01, max=0.60, step=0.01,readout=True,readout_format='5.3f'),
         x_search = widgets.FloatSlider(value=1,min=1, max=10000, step=1,readout=True),
         max_r = widgets.FloatSlider(value=1000,min=10, max=10000, step=10,readout=True),
         max_s = widgets.FloatSlider(value=10,min=1, max=50, step=1,readout=True))

interactive(children=(FloatSlider(value=3600.0, description='t', max=86400.0, min=60.0, step=60.0), FloatLogSl…

<function __main__.plot_s(t, T, S, SY, Q, b, x_search, max_s, max_r)>